In [1]:
import os

train_data_path = "./data/train_essays.csv"
train_prompts_path = "./data/train_prompts.csv"
supplement_data_dir = "./data/archive/"
supplement_data_files = [  os.path.join(supplement_data_dir,f)  
                          for f in os.listdir(supplement_data_dir)
                          if(f.endswith('.csv'))]

In [2]:
def set_proxy():
    import os
    cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # 代理
    os.environ['http_proxy'] = 'http://127.0.0.1:7890'
    os.environ['https_proxy'] = 'http://127.0.0.1:7890'
    os.environ['no_proxy'] = '127.0.0.1,localhost'
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['NO_PROXY'] = '127.0.0.1,localhost'
    return
set_proxy()

In [3]:
#!curl huggingface.co

In [4]:
import pandas as pd
train_data=pd.read_csv(train_data_path)
train_prompts = pd.read_csv(train_prompts_path)
instructions = {
    0:train_prompts['instructions'][0],
    1:train_prompts['instructions'][1],
}

train_data['prompt'] = train_data.apply(
    lambda r: instructions[r['prompt_id']] if  r['prompt_id'] in instructions else -1,axis=1
)


In [5]:
train_data

,id,prompt_id,text,generated,prompt
0,0059830c,0,Cars. Cars have been around since they became ...,0,Write an explanatory essay to inform fellow ci...
1,005db917,0,Transportation is a large necessity in most co...,0,Write an explanatory essay to inform fellow ci...
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0,Write an explanatory essay to inform fellow ci...
3,00940276,0,How often do you ride in a car? Do you drive a...,0,Write an explanatory essay to inform fellow ci...
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0,Write an explanatory essay to inform fellow ci...
...,...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0,Write a letter to your state senator in which ...
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0,Write an explanatory essay to inform fellow ci...
1375,ffa247e0,0,There's a new trend that has been developing f...,0,Write an explanatory essay to inform fellow ci...
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0,Write an explanatory essay to inform fellow ci...


In [6]:


supplement_train_data = pd.concat([
    pd.read_csv(f)
    for f in supplement_data_files
])


In [7]:
supplement_train_data

,text,label,source,fold,essay_id,prompt
0,There are alot reasons to keep our the despise...,0,persuade_corpus,2,NaN,NaN
1,Driving smart cars that drive by themself has ...,0,persuade_corpus,4,NaN,NaN
2,"Dear Principal,\n\nI believe that students at ...",0,persuade_corpus,0,NaN,NaN
3,"Dear Principal,\n\nCommunity service should no...",0,persuade_corpus,0,NaN,NaN
4,My argument for the development of the driverl...,0,persuade_corpus,3,NaN,NaN
...,...,...,...,...,...,...
39780,"""Oh man I didn't make the soccer team!"", yelle...",0,persuade_corpus,4,F7341069C4A4,NaN
39781,I believe that using this technology could be ...,0,persuade_corpus,2,AFE6E553DAC2,NaN
39782,The Face on Mars is a fascinating phenomenon t...,1,falcon_180b_v1,2,falcon_180b_v1_600,You have read the article 'Unmasking the Face ...
39783,Texting & Driving\n\nUsing your phone while dr...,0,persuade_corpus,2,A5F84C104693,NaN


In [8]:
supplement_train_data['generated'] = supplement_train_data['label']

supplement_train_data = supplement_train_data[['text','generated','prompt']]

In [9]:
supplement_train_data[supplement_train_data.generated == 1]

,text,generated,prompt
7,"As an eighth-grade student, I have noticed tha...",1,NaN
10,Studying Science and History at Generic_School...,1,NaN
17,"As an artist, I am constantly developing and h...",1,NaN
22,After conducting extensive research on the eff...,1,NaN
24,The COVID-19 pandemic has brought about a sign...,1,NaN
...,...,...,...
39767,Sure thing! Here's my attempt at writing an es...,1,Task: Research why it is beneficial for people...
39770,I think our principal's idea of making us do e...,1,Your principal has decided that all students m...
39771,I think it's a good idea for schools to have o...,1,Some schools have implemented policies that al...
39778,Advantages of Limiting Car Usage\n\nLimiting c...,1,NaN


In [10]:
import gc
#del train_data_re
gc.collect()
train_data_re = pd.DataFrame(train_data[['text','generated','prompt']])


In [11]:
train_data_all = pd.concat([train_data,supplement_train_data])

In [12]:
train_data_all.generated.value_counts()

generated
0    116747
1     44087
Name: count, dtype: int64

## build base_model

In [13]:
cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import AutoConfig, LlamaConfig 
# from llava.model.language_model.llava_llama import LlavaLlamaForCausalLM
# class LlavaConfig(LlamaConfig):
#     model_type = "llava"
# AutoConfig.register("llava", LlavaConfig)
model_name = "bigscience/bloom-3b"
#model_name = "ChocoWu/nextgpt_7b_tiva_v0"#"liuhaotian/llava-v1.5-7b"
original_model = AutoModelForCausalLM.from_pretrained(model_name,cache_dir=cache_dir,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
# tokenizer.add_tokens(["[PAD]"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## dataset

In [14]:
#train_data_re

In [15]:
def generate_prompt(prompt,feature_text):
    """
    
    """
    text= None 
    if(prompt is not None):
        template_with_prompt = f"""
        a prompt is shown as follows: 
        {prompt}.
        text generated by this this prompt is shown as below:
        {feature_text}.
        is this text generated by AI?
        """
        text = template_with_prompt
    else:
        template_without_prompt = f"""
        text generated without any prompt is shown as below:
        {feature_text}.
        is this text generated by AI?
        """
        text = template_without_prompt

    return text

In [16]:

from torch.utils.data import TensorDataset,DataLoader,RandomSampler
import torch

class myDataset(TensorDataset):

    def __init__(self, datalist,max_length=256,tokenizer=None,preprocess_func= None) -> None:
        super(myDataset,self).__init__()

        if(isinstance(datalist,pd.DataFrame)):
            self.datalist = datalist.to_dict(orient='list')
        elif(isinstance(datalist,dict)):
            self.datalist = datalist
        else:
            raise Exception("错误输入类型")
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.preprocess_func = preprocess_func

    def __len__(self):
        return len(self.datalist['text'])
    
    def __getitem__(self, index):
        text = self.datalist['text'][index]  
        prompt = self.datalist['prompt'][index]  

        final_text = self.preprocess_func(prompt,text)
        input_ids = self.tokenizer.encode(final_text)
        att_mask = [1] * len(input_ids)


        labels = None
        if('generated' in self.datalist):
            generated = self.datalist['generated'][index]
            label_text = " the text is generated by AI" if generated > 0 else "the text is written by students"
            label_ids = self.tokenizer.encode(label_text)

            labels = [self.tokenizer.eos_token_id]  * len(input_ids)
            labels = labels + label_ids
            input_ids = input_ids + label_ids   
            att_mask = [1] * len(input_ids)
        act_len = len(input_ids)
        while(len(input_ids) < self.max_length):
            input_ids.append(self.tokenizer.eos_token_id)
            if(labels is not None):
                labels.append(self.tokenizer.eos_token_id)
            att_mask.append(0)
        input_ids = input_ids[:self.max_length]
        labels = labels[:self.max_length]
        att_mask = att_mask[:self.max_length]
        if(labels is not None):
            return {'input_ids':torch.LongTensor(input_ids),'labels':labels,'att_mask':torch.LongTensor(att_mask),'act_len':act_len}
        else:
            return {'input_ids':torch.LongTensor(input_ids),'att_mask':torch.LongTensor(att_mask)}

        
dataset = myDataset(datalist=train_data_all,max_length=4096,tokenizer=tokenizer,preprocess_func=generate_prompt) 




In [17]:
train_data_all

,id,prompt_id,text,generated,prompt
0,0059830c,0.0,Cars. Cars have been around since they became ...,0,Write an explanatory essay to inform fellow ci...
1,005db917,0.0,Transportation is a large necessity in most co...,0,Write an explanatory essay to inform fellow ci...
2,008f63e3,0.0,"""America's love affair with it's vehicles seem...",0,Write an explanatory essay to inform fellow ci...
3,00940276,0.0,How often do you ride in a car? Do you drive a...,0,Write an explanatory essay to inform fellow ci...
4,00c39458,0.0,Cars are a wonderful thing. They are perhaps o...,0,Write an explanatory essay to inform fellow ci...
...,...,...,...,...,...
39780,NaN,NaN,"""Oh man I didn't make the soccer team!"", yelle...",0,NaN
39781,NaN,NaN,I believe that using this technology could be ...,0,NaN
39782,NaN,NaN,The Face on Mars is a fascinating phenomenon t...,1,You have read the article 'Unmasking the Face ...
39783,NaN,NaN,Texting & Driving\n\nUsing your phone while dr...,0,NaN


In [18]:
#all_len = [ dataset[i]['act_len'] for i in range(len(dataset))]

In [19]:
# import numpy as np
# cnt = 1000000000000
# np.percentile(all_len,q=99.9)

In [20]:
random_sampler = RandomSampler(dataset)

In [21]:

dataloader_random = DataLoader(dataset, batch_size=2, sampler=random_sampler)


In [22]:
import torch

In [23]:
#model(input_ids = torch.LongTensor([dataset[0]['input_ids']]),attention_mask=torch.LongTensor([dataset[0]['att_mask']]))

In [25]:
original_model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

## model definition

In [28]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from peft import LoraConfig, TaskType
lora_target_modules = ["q_proj","k_proj","v_proj",'o_proj']


peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, 
                         inference_mode=False, target_modules=lora_target_modules,
                         r=4, lora_alpha=16, lora_dropout=0.1)
model = get_peft_model(original_model,peft_config)
print(model.print_trainable_parameters())

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.04703666202518836
None


In [33]:
model = model.cuda()

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 23.90 GiB total capacity; 23.38 GiB already allocated; 52.38 MiB free; 23.38 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [32]:
import torch 
loss_fn = torch.nn.CrossEntropyLoss(reduction='mean',ignore_index=tokenizer.pad_token_id)



In [31]:
import torch
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(dataloader_random) ),
)

In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader_random)
    for step, batch in enumerate(pbar):
        batch = {k: v.cuda() for k, v in batch.items()}
        #print(batch)
        #outputs = model(batch['input_ids'],labels=batch['labels'])
        labels_tensor = batch['labels']
        outputs = model(batch['input_ids'])
        
        logits = outputs.logits
        logits = logits[...,:-1,:].contiguous()
        labels_tensor = labels_tensor[...,1:].contiguous()
        
        
        #loss = outputs.loss
        loss = loss_fn(logits.view(-1, logits.size(-1)), labels_tensor.view(-1))
    
        total_loss += loss.detach().float()
        #print(loss)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        if(step % 10 == 0):
            pbar.set_description(f"step {step} loss {loss}")